In [2]:
import schedule
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from datetime import datetime, timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as mtick
import matplotlib.dates as mdates

import requests

from selenium.webdriver.common.alert import Alert

In [3]:
username = "punyaruk.m@bgrimmpower.com"
password = "Pun11111*"

driver =  webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = "https://gsm.pttplc.com/f_GasQualityPoint.aspx?iszone=Hz2CeyMQb1OZH2wL5dGiJg%3d%3d&nzone=gf8njTYh2JMgStvkkQs26A%3d%3d&sRefPoint=A9LI5ZAo4QpdSBTb42Uoog%3d%3d&nperiod=105&Date=10-12-2023&OldDate=0#SECTION_02"
driver.get(url)
driver.maximize_window() #Maximize Chrome windows
time.sleep(10)

login = driver.find_element(By.XPATH, '/html/body/form/div[4]/div[1]/div[1]/div[1]/div/div[3]/button[2]')
login.click()
time.sleep(5)


user = driver.find_element(By.XPATH, '/html/body/form/div[5]/div[1]/div[1]/div[1]/div[3]/div/input')
pas = driver.find_element(By.XPATH, '/html/body/form/div[5]/div[1]/div[1]/div[1]/div[4]/div/input')
time.sleep(5)

user.send_keys(username)
time.sleep(5)
pas.send_keys(password+Keys.ENTER)


time.sleep(10)
cooki = driver.find_element(By.XPATH, "/html/body/div[2]/div[3]/div/div[1]/div/div[2]/div/button[2]")
cooki.click()


time.sleep(5)
service = driver.find_element(By.XPATH,'/html/body/form/div[4]/div[1]/div[2]/ul[1]/li[3]/a')
service.click()

time.sleep(5)
online_q = driver.find_element(By.XPATH, '/html/body/form/div[4]/div[2]/div[1]/div[2]/div/div/div/div[3]/a')
online_q.click()

time.sleep(5)
nrmr = driver.find_element(By.XPATH, "/html/body/form/div[4]/div[2]/div[2]/div[1]/div/div[2]/div[1]/a[16]")
nrmr.click()

In [ ]:
def task():
    try:
        #Data Preparation
        data_list=pd.read_html(driver.page_source)
        data_list[0]['Date - Time'] = pd.to_datetime(data_list[0]['Date - Time'], format='%d/%m/%Y %H:%M')
        df = data_list[0].copy()
        df['WI diff'] = df['WI (BTU/SCF)'].shift(1)-df['WI (BTU/SCF)'] 
        df['WI diff'] = df['WI diff'].abs()
        df.dropna(axis = 0 ,inplace=True)
        df=df.reset_index(drop=True)
        plt.figure(figsize=(10,5))
        plt.plot(df['Date - Time'].iloc[:50],df['WI (BTU/SCF)'].iloc[:50],"o-",linewidth=3)


        plt.title('WI (BTU/SCF) Gas Monitoring at NMR')
        plt.ylabel('WI (BTU/SCF)')

        dtFmt = mdates.DateFormatter('%d%b | %H:%M') # define the formatting
        plt.gca().xaxis.set_major_formatter(dtFmt) 
        plt.grid(True)

        plt.ylim(df['WI (BTU/SCF)'][0]-50,df['WI (BTU/SCF)'][0]+50)
        plt.xticks(rotation=30)
        plt.tight_layout()

        plt.savefig('PTT_WI.png')
        
        token = "cwEsMAOS7JeTRxFy8ks0CdtGfvP0eL5T3NZPkzsa8lz"
        url = "https://notify-api.line.me/api/notify"
        message = " "
        headers = {
            'Authorization': f"Bearer {token}",
        }
        with open(r'C:\\Users\punyaruk.m\\Desktop\\Task\\PTT_WI.png', 'rb') as image_file:
            # Create a dictionary for the message and file
            data = {
                'message': message,
            }

            # Send the request with the image file
            response = requests.post(url, headers=headers, data=data, files={'imageFile': image_file})
        
        #Create Alarm if deviation of WI between time >5 
        df1 = df.iloc[0:12]
        for x in df1[df1['WI diff'] > 5].index.tolist() :
            Time1= df1['Date - Time'][x]
            if x == 0:
                Time2 = Time1 + timedelta(minutes=5)
            else:
                Time2= df1['Date - Time'][x-1]
            WI = df1['WI diff'][x]
            message = f"Alarm! : \n{Time1} To {Time2}\nDifference between this time is {WI:.2f}"
        
            headers = {
                    'Authorization': f"Bearer {token}",
                      }
            
            data = {
                   'message': message,
                   }
            response = requests.post(url, headers=headers, data=data)
    except Exception as e: 
        print(f"Error: {e}")
        
schedule.every().hour.at(":00").do(task)

while True:
    schedule.run_pending()
    time.sleep(60)
#task()

C:\Users\punyaruk.m\AppData\Local\Temp\ipykernel_4040\534278585.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(10,5))
